In [1]:
import pandas as pd
from scipy.spatial.distance import cosine

data = pd.read_csv('lastfm-matrix-germany.csv')

data_germany = data.drop('user',1)

print data_germany.columns

data_ibs = pd.DataFrame(index=data_germany.columns,columns=data_germany.columns)


Index([u'a perfect circle', u'abba', u'ac/dc', u'adam green', u'aerosmith',
       u'afi', u'air', u'alanis morissette', u'alexisonfire', u'alicia keys',
       ...
       u'timbaland', u'tom waits', u'tool', u'tori amos', u'travis',
       u'trivium', u'u2', u'underoath', u'volbeat', u'yann tiersen'],
      dtype='object', length=285)


In [2]:
# Lets fill in those empty spaces with cosine similarities
# Loop through the columns
for i in range(0,len(data_ibs.columns)) :
    # Loop through the columns for each column
    for j in range(0,len(data_ibs.columns)) :
      # Fill in placeholder with cosine similarities
      data_ibs.ix[i,j] = 1-cosine(data_germany.ix[:,i],data_germany.ix[:,j])


print data_ibs

                      a perfect circle       abba      ac/dc adam green  \
a perfect circle                     1          0  0.0179172  0.0515539   
abba                                 0          1  0.0522788  0.0250706   
ac/dc                        0.0179172  0.0522788          1   0.113154   
adam green                   0.0515539  0.0250706   0.113154          1   
aerosmith                    0.0627765  0.0610563   0.177153  0.0566365   
afi                                  0          0  0.0678942          0   
air                          0.0517549  0.0167789  0.0757299  0.0933859   
alanis morissette            0.0607177  0.0295269  0.0380762          0   
alexisonfire                         0          0          0          0   
alicia keys                          0          0  0.0883332  0.0254164   
all that remains               0.13012          0  0.0203997          0   
amon amarth                  0.0429339          0   0.107696  0.0193673   
amy macdonald            

In [3]:
# Create a placeholder items for closes neighbours to an item
data_neighbours = pd.DataFrame(index=data_ibs.columns, columns=range(1, 11))

# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0, len(data_ibs.columns)):
    data_neighbours.ix[i, :10] = data_ibs.ix[0:, i].order(ascending=False)[:10].index

    # --- End Item Based Recommendations --- #
    
print data_neighbours.head(6).ix[:6,2:4]

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: order is deprecated, use sort_values(...)


                                      2                3              4
a perfect circle                   tool            dredg       deftones
abba                            madonna  robbie williams  elvis presley
ac/dc             red hot chili peppers        metallica    iron maiden
adam green               the libertines      the strokes   babyshambles
aerosmith                            u2     led zeppelin      metallica
afi                funeral for a friend     rise against   fall out boy


In [4]:

# --- Start User Based Recommendations --- #
 
# Helper function to get similarity scores
def getScore(history, similarities):
   return sum(history*similarities)/sum(similarities)

# Create a place holder matrix for similarities, and fill in the user name column
data_sims = pd.DataFrame(index=data.index,columns=data.columns)
data_sims.ix[:,:1] = data.ix[:,:1]

print data_sims

       user a perfect circle abba ac/dc adam green aerosmith  afi  air  \
0         1              NaN  NaN   NaN        NaN       NaN  NaN  NaN   
1        33              NaN  NaN   NaN        NaN       NaN  NaN  NaN   
2        42              NaN  NaN   NaN        NaN       NaN  NaN  NaN   
3        51              NaN  NaN   NaN        NaN       NaN  NaN  NaN   
4        62              NaN  NaN   NaN        NaN       NaN  NaN  NaN   
5        75              NaN  NaN   NaN        NaN       NaN  NaN  NaN   
6       130              NaN  NaN   NaN        NaN       NaN  NaN  NaN   
7       141              NaN  NaN   NaN        NaN       NaN  NaN  NaN   
8       144              NaN  NaN   NaN        NaN       NaN  NaN  NaN   
9       150              NaN  NaN   NaN        NaN       NaN  NaN  NaN   
10      205              NaN  NaN   NaN        NaN       NaN  NaN  NaN   
11      247              NaN  NaN   NaN        NaN       NaN  NaN  NaN   
12      256              NaN  NaN   Na

In [5]:
#Loop through all rows, skip the user column, and fill with similarity scores
for i in range(0,len(data_sims.index)):
    for j in range(1,len(data_sims.columns)):
        user = data_sims.index[i]
        product = data_sims.columns[j]
 
        if data.ix[i][j] == 1:
            data_sims.ix[i][j] = 0
        else:
            product_top_names = data_neighbours.ix[product][1:10]
            product_top_sims = data_ibs.ix[product].order(ascending=False)[1:10]
            user_purchases = data_germany.ix[user,product_top_names]
 
            data_sims.ix[i][j] = getScore(user_purchases,product_top_sims)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:11: FutureWarning: order is deprecated, use sort_values(...)


In [6]:
# Get the top songs
data_recommend = pd.DataFrame(index=data_sims.index, columns=['user','1','2','3','4','5','6'])
data_recommend.ix[0:,0] = data_sims.ix[:,0]

In [7]:
# Instead of top song scores, we want to see names
for i in range(0,len(data_sims.index)):
    data_recommend.ix[i,1:] = data_sims.ix[i,:].order(ascending=False).ix[1:7,].index.transpose()

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: order is deprecated, use sort_values(...)
  app.launch_new_instance()


In [8]:
# Print a sample
print data_recommend.ix[:10,:4]

   user                      1                      2                3
0     1         flogging molly               coldplay        aerosmith
1    33  red hot chili peppers          kings of leon        peter fox
2    42                 oomph!            lacuna coil        rammstein
3    51            the subways              the kooks  franz ferdinand
4    62           jack johnson                incubus       mando diao
5    75             hoobastank             papa roach           sum 41
6   130      alanis morissette  the smashing pumpkins        pearl jam
7   141           machine head        sonic syndicate          caliban
8   144                editors              nada surf      the strokes
9   150                placebo            the subways     eric clapton
10  205             in extremo          nelly furtado        finntroll
